In [1]:
import os

In [2]:
%pwd

'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver'

In [5]:
import dagshub
dagshub.init(repo_owner='anandsr724', repo_name='Handwritten_Equation_Solver', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as anandsr724

Initialized MLflow to track repo "anandsr724/Handwritten_Equation_Solver"

Repository anandsr724/Handwritten_Equation_Solver initialized!

2024/11/06 13:13:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run luxuriant-roo-801 at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0/runs/78e6389a058f4b2abf145e9b123623ea.
2024/11/06 13:13:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0.


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    classes_ideal: list


In [9]:
from equation_solver.constants import *
from equation_solver.utils.common import read_yaml, create_directories, save_json
from equation_solver.utils.common import get_unique_class_names


In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        training = self.config.training
        # prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "symbol_data")
        classes_ideal = get_unique_class_names(training_data)

        eval_config = EvaluationConfig(
            path_of_model=Path(training.trained_model_path),
            training_data=Path(training_data),
            mlflow_uri="https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow",
            all_params=self.params,
            params_image_size=params.IMAGE_SIZE,
            params_batch_size=params.BATCH_SIZE,
            classes_ideal=classes_ideal
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import numpy as np
from sklearn.model_selection import train_test_split


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.class_indices = {name: idx for idx, name in enumerate(self.config.classes_ideal)}
        self.model = self.load_model(self.config.path_of_model)

    def load_data(self):
        all_files = []
        for root, dirs, files in os.walk(self.config.training_data):
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg')):
                    class_name = os.path.basename(root)
                    if class_name in self.class_indices:
                        file_path = os.path.join(root, file)
                        all_files.append((file_path, class_name))

        # Shuffle the files
        np.random.shuffle(all_files)

        # Split file paths and labels
        file_paths, class_names = zip(*all_files)
        labels = [self.class_indices[class_name] for class_name in class_names]

        # Perform stratified split
        file_paths_train, file_paths_val, labels_train, labels_val = train_test_split(
            file_paths, labels, test_size=0.2, stratify=labels, random_state=34
        )

        # Create datasets
        val_ds = tf.data.Dataset.from_tensor_slices((file_paths_val, labels_val))

        # Apply preprocessing
        val_ds = val_ds.map(self.load_image, num_parallel_calls=tf.data.AUTOTUNE)

        # Batch
        val_ds = val_ds.cache().batch(self.config.params_batch_size).prefetch(tf.data.AUTOTUNE)

        return val_ds

    def load_image(self, file_path, label):
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, self.config.params_image_size[:-1])
        image = tf.image.rgb_to_grayscale(image)
        return image, label

    def load_model(self, path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.val_ds = self.load_data()
        self.score = self.model.evaluate(self.val_ds)
        self.save_score()


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="Model768")
            else:
                mlflow.tensorflow.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-11-06 13:13:49,848: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 13:13:49,851: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-06 13:13:49,852: INFO: common: created directory at: artifacts]

[2024-11-06 13:13:50,549: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
4/4 [==============================] - 1s 40ms/step - loss: 0.1589 - accuracy: 0.9880
[2024-11-06 13:13:50,854: INFO: common: json file saved at: scores.json]


2024/11/06 13:13:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\anand\AppData\Local\Temp\tmp85g_qg6a\model\data\model\assets
[2024-11-06 13:13:53,344: INFO: builder_impl: Assets written to: C:\Users\anand\AppData\Local\Temp\tmp85g_qg6a\model\data\model\assets]


2024/11/06 13:14:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Model768' already exists. Creating a new version of this model...
2024/11/06 13:14:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Model768, version 10
Created version '10' of model 'Model768'.
2024/11/06 13:14:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run unleashed-sheep-96 at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0/runs/cb67689961804422b62a328a76d4439d.
2024/11/06 13:14:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0.
